In [1]:
import os
import re
import time
import datetime
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from IPython.display import clear_output
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def foreigner(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('p')
    # print(links)
    # <span itemprop="birthPlace">

    links = soup.find_all(['span'] , attrs = { 'itemprop' : 'birthPlace'})
    for link in links:
        for c in (link.descendants):
            aa = ((c.find_next('a')).attrs)
            if 'country=US' in aa['href']:
                return False
    return True

## Should exccute the chromedriver and get the page source

In [3]:
def league(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
            
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_all_league' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
    return result

## Should exceute the chromedriver and get the page source

In [4]:
def allstar(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_allstar' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
            
    result = list(map (lambda x : str(int(x.split()[0])-1 ) + '-' + (x.split()[0][2:])  , result))
    return result

In [5]:
def Hall(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('li' , {'class'  : 'bling_special bling_hof'})
    for link in links:
        if link.text == "Hall of Fame":
            return True

In [6]:
def btn_click(wd,url):
    wd.get(url)
    script = "vjs_addClass(document.getElementById('leaderboard_allstar'),'show_all')"
    script2 = "vjs_addClass(document.getElementById('leaderboard_allleague'),'show_all')"
    btn = wd.execute_script(script)
    btn = wd.execute_script(script2)
    return(wd.page_source)

In [7]:
def get_all_info(wd,url):
    html = btn_click(wd,url)
    result = ([Hall(html) ,league(html), allstar(html), foreigner(html)])
    return (result)

In [8]:
all_data = []
df = pd.read_csv('player_my_df.csv')
info = pd.read_excel('info.xlsx')

dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

In [9]:

# # chrome_options.add_argument("--headless") # make the chrome invisible
# chrome_options = Options() 
# chrome_options.add_argument("--window-size=100x10")
# wd = webdriver.Chrome(options=chrome_options)

# # empty = pd.Series(['**' ,'**',['**'],['**']])
# empty = (['**' ,'**',['**'],['**']])
# df_remained = pd.DataFrame()
# none = pd.DataFrame([None,[],[],False])

# df = pd.DataFrame(df)
# row, col = df.shape

# all_url = (df['5'])
# for i in range(4044, len(all_url)):
#     print(i,dirs_no[0],len(all_data))
#     if len(dirs_no) >= 0:
#         if dirs_no[0] != i:
#             #df_remained.append(empty,ignore_index=True)
#             all_data.append(empty)
#             continue
#         else:
#             dirs_no.pop(0)
        
#     url = all_url[i]
#     appended_data = get_all_info(wd,url)
#     all_data.append(appended_data)
    
#     if i % 100 == 0:
#         print(i)
#         clear_output()
#         df_remained.to_excel('remain.xlsx') 

In [10]:
# chrome_options = Options() 
# chrome_options.add_argument("--window-size=100x10")
# wd = webdriver.Chrome(options=chrome_options)

# sample_url = "https://www.basketball-reference.com/players/g/greenac01.html"
# get_all_info(wd,sample_url)

## Load the all data processed

In [11]:
import pickle
dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

with open ('all_data', 'rb') as fp:
    data = pickle.load(fp)
      
info = pd.read_excel('info.xlsx')

df = pd.DataFrame()   
row = len(data)
ind = info.columns

for i in range(row):
    if(len(dirs_no) > 0):
        if i != dirs_no[0]:
            continue
        dirs_no.pop(0)
        
    my_list = pd.read_html('data/' + str(i) + '.xls')
    r , c = ((my_list[0].shape))
#     print(r,c)
    
    if c != 30:
        print(i)
        continue
    
    small_df = pd.DataFrame(np.array(my_list[0]).reshape(r , c))
    small_df = small_df.dropna(axis=1)

    ## Process fix award ##
    ## [None, ['1988-89'], ['1989-90'], False]
    award = data[i]
    inf = list(info.iloc[i,:])

    
    if(award[0]):
        small_df['名人堂']=(pd.Series([1]*r))
    else:
        small_df['名人堂']= (pd.Series([0]*r))

    if(award[3]):
        small_df['外籍球員']=(pd.Series([1]*r))
    else:
        small_df['外籍球員']= (pd.Series([0]*r))

    small_df['是否入圍全明星'] = pd.Series([0]*r)
    small_df['當年度獎項次數'] = pd.Series([0]*r)
    small_df['當年度獎項YN'] = pd.Series([0]*r)
    
    #([Hall(html) ,league(html), allstar(html), foreigner(html)])
    
    all_series = list(small_df.iloc[:,0])
    
    for m in range(len(award[2])):
        for n in range(len(all_series)):
            if(award[2][m] == all_series[n]):
                small_df['是否入圍全明星'][n] = 1
    
    for m in range(len(award[1])):
        for n in range(len(all_series)):
            if(award[1][m] == all_series[n]):
                small_df['當年度獎項次數'][n] += 1
                small_df['當年度獎項YN'][n] = 1

    ## Process info ##
    for k in range(len(inf)):
        small_df[ind[k]]= pd.Series([inf[k]] * r)

    result = pd.concat([df,small_df],axis=0,ignore_index=True)

0
3
6
9
10
13
14
17
18
22
25
26
27
29
30
31
32
33
34
35
36
37
38
39
40
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
66
68
69
70
71
72
73
74
75
77
78
79
80
81
83
84
92
96
100
101
102
106
108
110
111
112
115
116
117
118
121
122
124
125
127
128
131
132
134
135
136


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


140
143
144
145
149
150
152
153
154
157
158
160
161
162
163
165
166
167
168
170
173
174
176
177
178
179
180
182
184
185
190
191
192
193
194
198
200
201
205
208
209
212
213
214
215
216
217
218
220
221
223
224
226
227
228
229
230
232
233
234
236
238
240
241
245
246
247
248
249
251
253
255
256
257
258
259
260
261
263
264
265
267
268
270
272
273
274
275
276
277
278
279
280
281
283
287
290
291
294
295
296
297
298
299
300
302
303
307
309
310
311
312
313
314
315
316
319
320
321
322
324
326
327
328
330
331
334
335
336
338
339
340
342
343
344
345
346
347
349
350
351
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
376
377
378
379
380
381
382
383
385
388
389
390
392
393
394
395
396
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
414
417
419
420
421
422
423
425
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
443
444
445
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
473
474
475
476
